In [5]:
import requests
from bs4 import BeautifulSoup
import locale

from tabulate import tabulate

#from modelos import FundoImobiliario, Estrategia

#locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')


def trata_porcentagem(porcentagem_str):
    return locale.atof(porcentagem_str.split('%')[0])


def trata_decimal(decimal_str):
    return locale.atof(decimal_str)


headers = {'User-Agent': 'Mozilla/5.0'}

resposta = requests.get('https://fundamentus.com.br/fii_resultado.php', headers=headers)

soup = BeautifulSoup(resposta.text, 'html.parser')

linhas = soup.find(id="tabelaResultado").find('tbody').findAll('tr')

resultado = []

estrategia = Estrategia(
    cotacao_atual_minima=50,
    dividiend_yield_minimo=5,
    p_vp_minimo=0.20,
    valor_mercado_minima=20000000,
    # liquidez_minima=5000,
    qt_minima_imoveis=3,
    maxima_vacancia_media=10
)

for linha in linhas:
    dados_fundo = linha.findAll('td')
    codigo = dados_fundo[0].text
    segmento = dados_fundo[1].text
    cotacao = trata_decimal(dados_fundo[2].text)
    ffo_yield = trata_porcentagem(dados_fundo[3].text)
    dividiend_yield = trata_porcentagem(dados_fundo[4].text)
    p_vp = trata_decimal(dados_fundo[5].text)
    valor_mercado = trata_decimal(dados_fundo[6].text)
    liquidez = trata_decimal(dados_fundo[7].text)
    qt_imoveis = int(dados_fundo[8].text)
    preco_m2 = trata_decimal(dados_fundo[9].text)
    aluguel_m2 = trata_decimal(dados_fundo[10].text)
    cap_rate = trata_porcentagem(dados_fundo[11].text)
    vacancia = trata_porcentagem(dados_fundo[12].text)

    fundo_imobiliario = FundoImobiliario(
        codigo, segmento, cotacao, ffo_yield, dividiend_yield, p_vp, valor_mercado, liquidez, qt_imoveis, preco_m2,
        aluguel_m2, cap_rate, vacancia
    )

    if estrategia.aplica_estrategia(fundo_imobiliario):
        resultado.append(fundo_imobiliario)

cabecalho = ["CÓDIGO", "SEGMENTO", "COTAÇÃO ATUAL", "DIVIDEND YIELD"]

tabela = []

for elemento in resultado:
    tabela.append([
        elemento.codigo,
        elemento.segmento,
        locale.currency(elemento.cotacao_atual),
        f'{locale.str(elemento.dividiend_yield)}%'
    ])

print(tabulate(tabela, headers=cabecalho, showindex='always', tablefmt='fancy_grid'))







ValueError: could not convert string to float: '9,38'

In [6]:
class FundoImobiliario:
    def __init__(self, codigo, segmento, cotacao_atual, ffo_yield, dividiend_yield, p_vp, valor_mercado, liquidez,
                 qt_imoveis, preco_m2, aluguel_m2, cap_rate, vacancia_media):
        self.codigo = codigo
        self.segmento = segmento
        self.cotacao_atual = cotacao_atual
        self.ffo_yield = ffo_yield
        self.dividiend_yield = dividiend_yield
        self.p_vp = p_vp
        self.valor_mercado = valor_mercado
        self.liquidez = liquidez
        self.qt_imoveis = qt_imoveis
        self.preco_m2 =preco_m2
        self.aluguel_m2 = aluguel_m2
        self.cap_rate = cap_rate
        self.vacancia_media = vacancia_media

class Estrategia:
    def __init__(self, segmento="", cotacao_atual_minima=0, ffo_yield_minimo=0, dividiend_yield_minimo=0, p_vp_minimo=0,
                 valor_mercado_minima=0, liquidez_minima=0, qt_minima_imoveis=0, valor_minimo_preco_m2=0,
                 valor_minimo_aluguel_m2=0, valor_minimo_cap_rate=0, maxima_vacancia_media=0):
        self.segmento = segmento
        self.cotacao_atual_minima = cotacao_atual_minima
        self.ffo_yield_minimo = ffo_yield_minimo
        self.dividiend_yield_minimo = dividiend_yield_minimo
        self.p_vp_minimo = p_vp_minimo
        self.valor_mercado_minima = valor_mercado_minima
        self.liquidez_minima = liquidez_minima
        self.qt_minima_imoveis = qt_minima_imoveis
        self.valor_minimo_preco_m2 =valor_minimo_preco_m2
        self.valor_minimo_aluguel_m2 = valor_minimo_aluguel_m2
        self.valor_minimo_cap_rate = valor_minimo_cap_rate
        self.maxima_vacancia_media = maxima_vacancia_media

    def aplica_estrategia(self, fundo: FundoImobiliario):
        if self.segmento != "":
            if fundo.segmento != self.segmento:
                return False

        if fundo.cotacao_atual < self.cotacao_atual_minima \
                or fundo.ffo_yield < self.ffo_yield_minimo \
                or fundo.dividiend_yield < self.dividiend_yield_minimo \
                or fundo.p_vp < self.p_vp_minimo \
                or fundo.valor_mercado < self.valor_mercado_minima \
                or fundo.liquidez < self.liquidez_minima \
                or fundo.qt_imoveis < self.qt_minima_imoveis \
                or fundo.preco_m2 < self.valor_minimo_preco_m2 \
                or fundo.aluguel_m2 < self.valor_minimo_aluguel_m2 \
                or fundo.cap_rate < self.valor_minimo_cap_rate \
                or fundo.vacancia_media < self.maxima_vacancia_media:
            return False
        else:
            return True

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl

class fii:
    def __init__(self, ticker):
        self.ticker = ticker
    def pegar_dividendos_fii(self):
        agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
        url = f"https://www.fundamentus.com.br/fii_proventos.php?papel={self.ticker}&tipo=2"
        webpage = requests.get(url, headers=agent)
        soup = BeautifulSoup(webpage.content, "html.parser")
        resultados = soup.find(id = "resultado")
        tabela_body = resultados.find("tbody")
        linhas = tabela_body.find_all("tr")
        data = []
        for linha in linhas:
            colunas = linha.find_all("td")
            colunas = [ele.text.strip() for ele in colunas]
            data.append([ele for ele in colunas if ele])
        data_output = pd.DataFrame(data, columns=["Data_com", "rendimento", "data_rendimento", "valor"])
        data_output['papel'] = self.ticker
        # aqui pegaremos sempre o ultimo dividendo disponivel
        data_output = data_output.iloc[0]

        return data_output

def get_dividend(papel):
    return fii(papel).pegar_dividendos_fii()
 
# usando map para rodar a funcao para os tickers
resultados_map = pd.DataFrame(list(map(get_dividend, ["VGIP11", "BCRI11", "VINO11"])))

# exportando FIIs
#resultados_map.to_excel("dividendos_fiis.xlsx")
print(resultados_map)

     Data_com  rendimento data_rendimento valor   papel
0  13/04/2023  Rendimento      20/04/2023  1,10  VGIP11
0  31/03/2023  Rendimento      14/04/2023  0,86  BCRI11
0  31/03/2023  Rendimento      17/04/2023  0,31  VINO11
